In [23]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [24]:
data = pd.read_csv("output/songs_with_lyrics_cleaned.csv")
separated_lyrics = data['lyrics'].dropna()

corpus = []

print("🧹 Cleaning and Segmenting Lyrics...")

for lyric_text in separated_lyrics:
    if isinstance(lyric_text, str):
        text_no_tags = re.sub(r'\[.*?\]', ' ', lyric_text, flags=re.DOTALL)
        raw_segments = text_no_tags.split('\n')
        clean_segments = [s.strip().lower() for s in raw_segments if len(s.strip()) > 30]
        
        # Add to your main list
        corpus.extend(clean_segments)

print(f"✅ Processing complete. Total segments: {len(corpus)}")
# Now 'corpus' is ready for SBERT

🧹 Cleaning and Segmenting Lyrics...
✅ Processing complete. Total segments: 6762


In [29]:
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')
nltk.download('stopwords')

def get_top_keywords_filtered(texts, n=10):
    # 1. EXPANDED STOPWORDS (Grammar + Slang)
    custom_stops = list(nltk.corpus.stopwords.words('english'))
    pop_slang = [
        "lil", "gon", "bout", "em", "ayy", "uh", "huh", "ha", "vi", "doo", "wee",
        "yeah", "oh", "baby", "know", "like", "got", "just", "don", "ve", "ll", 
        "want", "need", "love", "time", "way", "make", "say", "come", "go", "right",
        "look", "good", "feel", "really", "cause", "wanna", "gonna", "gotta", "ain",
        "girl", "boy", "man", "woman", "hey", "ooh", "whoa", "shit", "fuck", "bitch",
        "nigga", "niggas", "damn", "ass", "tell", "think", "never", "back", "let",
        "swag", "yuh", "hum", "who", "what", "where", "why", "top", "call", "put",
        "gang", "thug", "bro", "pussy", "tryna", "chick", "girls", "slatt", "mmh"
    ]
    
    # 2. ABSTRACT NOUNS BLACKLIST (The "Non-Visual" Filter)
    # These are nouns/verbs that appear often but you can't build them in 3D.
    abstract_concepts = [
        "life", "day", "night", "heart", "mind", "world", "everything", "nothing", 
        "things", "nothin", "songs", "song", "name", "eyes", "face", "voice", 
        "head", "hand", "hands", "god", "soul", "mind", "pain", "hope", "wish", 
        "fame", "lie", "lies", "truth", "word", "words", "end", "reason", "part",
        "told", "saw", "knew", "met", "said", "made", "found", "came", "went", # Past tense verbs
        "die", "born", "live", "dead", "death", "control", "move", "wait", "hold",
        "stop", "start", "change", "keep", "leave", "stay", "believe", "remember"
    ]

    # Combine all lists
    all_stops = custom_stops + pop_slang + abstract_concepts

    # 3. VECTORIZE
    vec = CountVectorizer(stop_words=all_stops).fit(texts)
    bag_of_words = vec.transform(texts)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    
    # 4. FINAL CLEANUP (Noun Check + Length Check)
    cleaned_list = []
    for word, freq in sorted(words_freq, key=lambda x: x[1], reverse=True):
        if len(word) <= 2: continue # Remove short garbage
        
        # Verify it's a Noun (NN)
        pos_tag = nltk.pos_tag([word])[0][1]
        if pos_tag.startswith('NN'):
            cleaned_list.append((word, freq))
            
    return cleaned_list[:n]

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/Bay_Techatham/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Bay_Techatham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Bay_Techatham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
import optuna
from sklearn.metrics import silhouette_score

# 1. DEFINE THE OBJECTIVE FUNCTION
# Optuna will run this function many times with different 'k' values
def objective(trial):
    # Ask Optuna to try a K between 4 and 15
    k = trial.suggest_int('n_clusters', 4, 30)
    
    # Train the model
    model = KMeans(n_clusters=k, random_state=42, n_init=10)
    model.fit(embeddings)  # Use your existing SBERT embeddings
    
    # Calculate score (Higher is better)
    # Silhouette score measures how distinct the clusters are
    score = silhouette_score(embeddings, model.labels_)
    
    return score

# 2. RUN THE OPTIMIZATION
print("🤖 Starting Optuna Optimization...")
study = optuna.create_study(direction='maximize')  # We want MAX separation
study.optimize(objective, n_trials=20)  # Try 20 different times

# 3. PRINT RESULTS
print("-" * 40)
print(f"🏆 Best K found: {study.best_params['n_clusters']}")
print(f"📈 Best Silhouette Score: {study.best_value:.4f}")
print("-" * 40)

# 4. OPTIONAL: VISUALIZE
# optuna.visualization.plot_optimization_history(study).show()

[I 2026-01-03 16:51:22,645] A new study created in memory with name: no-name-53bf9b13-ff4b-45eb-bcd8-650a90a7657e


🤖 Starting Optuna Optimization...


[I 2026-01-03 16:51:26,826] Trial 0 finished with value: 0.02043554000556469 and parameters: {'n_clusters': 19}. Best is trial 0 with value: 0.02043554000556469.
[I 2026-01-03 16:51:29,953] Trial 1 finished with value: 0.01862427219748497 and parameters: {'n_clusters': 17}. Best is trial 0 with value: 0.02043554000556469.
[I 2026-01-03 16:51:34,094] Trial 2 finished with value: 0.01531270518898964 and parameters: {'n_clusters': 13}. Best is trial 0 with value: 0.02043554000556469.
[I 2026-01-03 16:51:37,912] Trial 3 finished with value: 0.02079150639474392 and parameters: {'n_clusters': 20}. Best is trial 3 with value: 0.02079150639474392.
[I 2026-01-03 16:51:42,682] Trial 4 finished with value: 0.019564403221011162 and parameters: {'n_clusters': 30}. Best is trial 3 with value: 0.02079150639474392.
[I 2026-01-03 16:51:45,105] Trial 5 finished with value: 0.012676121667027473 and parameters: {'n_clusters': 7}. Best is trial 3 with value: 0.02079150639474392.
[I 2026-01-03 16:51:49,010]

----------------------------------------
🏆 Best K found: 28
📈 Best Silhouette Score: 0.0231
----------------------------------------


In [33]:
print(f"\n🧠 Analyzing {len(corpus)} lyric segments with SBERT...")

NUM_CLUSTERS = 10

# A. Embed
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(corpus)

# B. Cluster
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
kmeans.fit(embeddings)
cluster_labels = kmeans.labels_

# ==========================================
# 4. RESULTS (THE "SHOPPING LIST")
# ==========================================
print("\n" + "="*40)
print("   YOUR VISUAL SHOPPING LIST")
print("="*40)

# Helper to find words from the clusters
def get_top_keywords(texts, n=8):
    vec = CountVectorizer(stop_words='english').fit(texts)
    bag_of_words = vec.transform(texts)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    return sorted(words_freq, key=lambda x: x[1], reverse=True)[:n]

df = pd.DataFrame({'text': corpus, 'cluster': cluster_labels})

for i in range(NUM_CLUSTERS):
    cluster_text = df[df['cluster'] == i]['text'].tolist()
    
    # Use the new function
    keywords = get_top_keywords_filtered(cluster_text)
    
    print(f"\n📁 CLUSTER {i+1} (Sample: {len(cluster_text)})")
    # Print just the words, not the frequency counts
    print(f"   Keywords: {[k[0] for k in keywords]}")
    print(f"   👉 DECISION: What 3D asset matches these nouns?")


🧠 Analyzing 6762 lyric segments with SBERT...

   YOUR VISUAL SHOPPING LIST

📁 CLUSTER 1 (Sample: 759)
   Keywords: ['talk', 'friends', 'money', 'thought', 'ones', 'kept', 'somebody', 'people', 'care', 'school']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 2 (Sample: 1075)
   Keywords: ['money', 'coupe', 'house', 'bag', 'home', 'round', 'light', 'hit', 'body', 'cut']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 3 (Sample: 695)
   Keywords: ['lucy', 'heaven', 'shake', 'diamonds', 'party', 'music', 'king', 'sky', 'stole', 'doin']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 4 (Sample: 831)
   Keywords: ['hate', 'try', 'learn', 'left', 'break', 'living', 'feels', 'hurt', 'drugs', 'half']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 5 (Sample: 328)
   Keywords: ['masa', 'rap', 'gettin', 'hoes', 'pop', 'wayne', 'playin', 'fuckin', 'money', 'chaos']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 6 (Sample: 447)
  

In [34]:
import pickle

with open('kmeans_presets.pkl', 'wb') as f:
    pickle.dump(kmeans, f)
print("✅ Model saved as 'kmeans_presets.pkl'")

# 2. SAVE YOUR ASSET MAP
# Update this dict based on your final 4-8 decisions
preset_map = {
    0: "room_environment",   # Cluster 1 (Friends/School)
    1: "urban_environment",  # Cluster 2 (Money/Coupe)
    2: "sky_environment",    # Cluster 3 (Heaven/Sky)
    3: "dark_environment",   # Cluster 4 (Hate/Drugs)
    4: "urban_environment",  # Cluster 5 (Rap/Chaos)
    5: "urban_environment",  # Cluster 6 (Ride/Crib)
    6: "room_environment",   # Cluster 7 (Lover/Tonight)
    7: "fire_environment",   # Cluster 8 (Sun/Fire)
    8: "snow_environment",   # Cluster 9 (Christmas/Snow)
    9: "urban_environment"   # Cluster 10 (Violence/City)
}

with open('preset_labels.pkl', 'wb') as f:
    pickle.dump(preset_map, f)
print("✅ Labels saved as 'preset_labels.pkl'")

✅ Model saved as 'kmeans_presets.pkl'
✅ Labels saved as 'preset_labels.pkl'
